# Spotify Tracks — ETL (Data Architect)
**Objective:** Load, clean, validate, and export an analysis-ready Spotify tracks dataset.

**Input:** `../data/spotify_tracks_dataset.csv`  
**Output:** set by you in `OUTPUT_CSV` (e.g., `data/spotify_tracks_clean_YYYYMMDD.csv`)

## Objectives
1. Setup & configurable paths
2. Load data
3. Quick scan (shape, columns, types, missing)
4. Cleaning (column names, dtypes, dedupe, simple imputations)
5. Light feature derivations (e.g., duration_min)
6. Quality checks (IDs unique, NaN checks, basic ranges)
7. Export cleaned dataset (filename you choose)
8. Notes & limitations

## 1. Importing Required Libraries
For this ETL process, we’ll use:
- **pandas**: for data loading, cleaning, and transformation  
- **numpy**: for numerical operations  
- **pathlib.Path**: for handling file paths in a platform-independent way

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

print("Libraries imported successfully.")

Libraries imported successfully.


# 2. Setup & configurable paths

- Keep the input in `data/spotify_tracks_dataset.csv` (or change the path below).
- Choose any output filename you like by editing `OUTPUT_CSV`.

In [3]:
from pathlib import Path
import numpy as np
import pandas as pd

# === Configurable paths ===
INPUT_CSV = Path("data/spotify_tracks_dataset.csv")   # <- your dataset lives in the separate 'data' folder
OUTPUT_CSV = Path("data/spotify_tracks_clean_1.csv")  # <- change this to any filename you want

print("Input file:", INPUT_CSV.resolve())
print("Output file (will be created):", OUTPUT_CSV.resolve())

Input file: /Users/nasraibrahim/Documents/vscode-projects/song-recommendation-dashboard/notebooks/data/spotify_tracks_dataset.csv
Output file (will be created): /Users/nasraibrahim/Documents/vscode-projects/song-recommendation-dashboard/notebooks/data/spotify_tracks_clean_1.csv


# 3. Load data

We'll load the Spotify tracks dataset.  
If the file is missing or the path is incorrect, you'll see a warning.

In [9]:
from pathlib import Path
import numpy as np
import pandas as pd

# === Configurable paths ===
# Use correct relative path from the 'notebooks' folder to the 'data' folder
INPUT_CSV = Path("../data/spotify_tracks_dataset.csv")   # <- your dataset lives in the separate 'data' folder
OUTPUT_CSV = Path("../data/spotify_tracks_clean_1.csv")  # <- change this to any filename you want

print("Input file:", INPUT_CSV.resolve())
print("Output file (will be created):", OUTPUT_CSV.resolve())

# === Load data ===
if not INPUT_CSV.exists():
    print(f"Warning: Could not find {INPUT_CSV.resolve()}. Check the folder path/name.")
    df_raw = None
else:
    df_raw = pd.read_csv(INPUT_CSV)
    display(df_raw.head(3))

Input file: /Users/nasraibrahim/Documents/vscode-projects/song-recommendation-dashboard/data/spotify_tracks_dataset.csv
Output file (will be created): /Users/nasraibrahim/Documents/vscode-projects/song-recommendation-dashboard/data/spotify_tracks_clean_1.csv


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.1430,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.166,...,-17.235,1,0.0763,0.9240,0.000006,0.101,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.359,...,-9.734,1,0.0557,0.2100,0.000000,0.117,0.120,76.332,4,acoustic


# 4. Quick scan

Quick scan (shape, columns, types, missing)
Understand structure to guide cleaning decisions.

In [10]:
print("Shape:", df_raw.shape)
print("\nColumns:", df_raw.columns.tolist())

print("\nInfo:")
print(df_raw.info())

print("\nMissing values by column (top 20):")
df_raw.isna().sum().sort_values(ascending=False).head(20)

Shape: (114000, 21)

Columns: ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64


artists             1
album_name          1
track_name          1
Unnamed: 0          0
mode                0
time_signature      0
tempo               0
valence             0
liveness            0
instrumentalness    0
acousticness        0
speechiness         0
key                 0
loudness            0
track_id            0
energy              0
danceability        0
explicit            0
duration_ms         0
popularity          0
dtype: int64

# 5. Cleaning 

**Goals**
- Standardise column names to `snake_case`.
- Fix dtypes (e.g., `explicit` → bool; `year` → numeric; dates → datetime).
- Deduplicate (prefer unique `id`; else a composite like `name + artists + album` if present).
- Handle missing values (simple, transparent rules).

In [11]:
def to_snake(name: str) -> str:
    return (
        name.strip()
            .replace(" ", "_")
            .replace("-", "_")
            .replace("/", "_")
            .lower()
    )

df = df_raw.copy()
df.columns = [to_snake(c) for c in df.columns]

# Track expected columns if they exist in your CSV version
expected_cols = [
    "id","name","artists","album","release_date","explicit","duration_ms","popularity",
    "danceability","energy","key","loudness","mode","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","time_signature","year"
]
present = [c for c in expected_cols if c in df.columns]
missing = [c for c in expected_cols if c not in df.columns]
print("Present expected cols:", present)
print("Missing expected cols:", missing)

# ---- Dtype fixes (only if columns exist) ----
if "explicit" in df.columns:
    # Convert strings/numbers to booleans
    df["explicit"] = (
        df["explicit"].astype(str).str.strip().str.lower().isin(["1","true","t","yes","y"])
    )

if "year" in df.columns:
    df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")

if "release_date" in df.columns:
    df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")

if "duration_ms" in df.columns:
    df["duration_ms"] = pd.to_numeric(df["duration_ms"], errors="coerce")

# Helper feature: duration in minutes if duration_ms exists
if "duration_ms" in df.columns and "duration_min" not in df.columns:
    df["duration_min"] = (df["duration_ms"] / 60000.0).round(2)

# Popularity as integer if present
if "popularity" in df.columns:
    df["popularity"] = pd.to_numeric(df["popularity"], errors="coerce").fillna(0).astype(int)

# ---- Deduplication ----
if "id" in df.columns:
    before = len(df)
    df = df.drop_duplicates(subset=["id"])
    print(f"Dedup by id: {before} -> {len(df)}")
else:
    # fall back to a reasonable composite if available
    keys = [c for c in ["name","artists","album"] if c in df.columns]
    if keys:
        before = len(df)
        df = df.drop_duplicates(subset=keys)
        print(f"Dedup by {keys}: {before} -> {len(df)}")

# ---- Missing values handling (simple & transparent) ----
audio_cols = [c for c in [
    "danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo"
] if c in df.columns]

# Drop rows where all audio features are missing
if audio_cols:
    before = len(df)
    df = df.dropna(subset=audio_cols, how="all")
    print(f"Dropped rows with all audio features missing: {before} -> {len(df)}")

# Median impute for remaining NaNs in numeric audio features (keeps it simple)
for c in audio_cols:
    if df[c].dtype.kind in "if" and df[c].isna().any():
        df[c] = df[c].fillna(df[c].median())

df_clean = df.copy()
print("Cleaned shape:", df_clean.shape)
df_clean.head(3)

Present expected cols: ['artists', 'explicit', 'duration_ms', 'popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
Missing expected cols: ['id', 'name', 'album', 'release_date', 'year']
Dedup by ['artists']: 114000 -> 31438
Dropped rows with all audio features missing: 31438 -> 31438
Cleaned shape: (31438, 22)


,unnamed:_0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,duration_min
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,0,0.1430,0.0322,0.000001,0.358,0.715,87.917,4,acoustic,3.84
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.166,...,1,0.0763,0.9240,0.000006,0.101,0.267,77.489,4,acoustic,2.49
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.359,...,1,0.0557,0.2100,0.000000,0.117,0.120,76.332,4,acoustic,3.51


# 6. Quality checks

- IDs unique (if `id` exists).
- No NaNs in the chosen audio features after cleaning.
- Basic range sanity on key numeric columns.

In [12]:
# 1) ID uniqueness
if "id" in df_clean.columns:
    n_rows = len(df_clean)
    n_unique = df_clean["id"].nunique()
    print(f"ID uniqueness — unique: {n_unique} / rows: {n_rows}")
    assert n_unique == n_rows, "IDs are not unique after cleaning."

# 2) NaNs in audio features
if audio_cols:
    total_nans = df_clean[audio_cols].isna().sum().sum()
    print("Total NaNs in audio features:", total_nans)
    assert total_nans == 0, "There are NaNs in audio features after cleaning."

# 3) Basic numeric sanity (optional)
num_cols = [c for c in df_clean.columns if str(df_clean[c].dtype) in ("int64","float64","Int64")]
desc = df_clean[num_cols].describe().T
desc[["mean","std","min","max"]].head(12)

Total NaNs in audio features: 0


,mean,std,min,max
unnamed:_0,52610.978752,33445.501615,0.000,113992.000
popularity,37.387461,20.261125,0.000,100.000
duration_ms,230674.416693,107541.174275,0.000,4563897.000
danceability,0.582363,0.175829,0.000,0.979
energy,0.639394,0.251491,0.000,1.000
key,5.364018,3.558598,0.000,11.000
loudness,-8.332680,5.399160,-49.307,4.532
mode,0.604714,0.488920,0.000,1.000
speechiness,0.092390,0.108708,0.000,0.965
acousticness,0.316812,0.331021,0.000,0.996


# 7. Export cleaned dataset

Now we'll export the cleaned DataFrame to a CSV file for downstream analysis. The output path is set in `OUTPUT_CSV` above.


In [13]:
# Export the cleaned DataFrame to CSV
df_clean.to_csv(OUTPUT_CSV, index=False)
print(f"Cleaned dataset exported to: {OUTPUT_CSV.resolve()}")

Cleaned dataset exported to: /Users/nasraibrahim/Documents/vscode-projects/song-recommendation-dashboard/data/spotify_tracks_clean_1.csv


# 8. Summary & Findings

- **Data Loading:** Successfully loaded the Spotify tracks dataset from the specified CSV file.
- **Initial Scan:** Inspected the dataset for shape, columns, data types, and missing values to understand its structure.
- **Cleaning:**  
  - Standardized column names to `snake_case`.
  - Converted data types (e.g., `explicit` to boolean, `year` to integer, `release_date` to datetime).
  - Removed duplicate records based on unique identifiers.
  - Handled missing values using simple, transparent rules (e.g., median imputation for audio features).
  - Derived new features such as `duration_min` from `duration_ms`.
- **Quality Checks:**  
  - Ensured all IDs are unique.
  - Verified no missing values remain in key audio feature columns.
  - Checked for reasonable ranges in numeric columns.
- **Export:**  
  - Exported the cleaned dataset to a new CSV file for downstream analysis.

## Limitations

- Cleaning rules are simple and may not cover all edge cases or data quality issues.
- Some columns may still contain outliers or unexpected values that require further domain-specific validation.
- Imputation uses median values, which may not be optimal for all features or analyses.
- The process assumes the input data structure is consistent; changes in the source schema may require code updates.

---

**Next Steps:**  
Consider further feature engineering, advanced outlier detection, or integrating additional data sources for deeper analysis.